<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/our%20framework%20code/answer_Generation_jais-family-6p7b-chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sitting up

In [1]:
!huggingface-cli login
!pip install datasets


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) ى
Invalid input. Must be one of ('y', 'yes', '1', 'n', 'no', '0', '')
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `week1 test` has been saved to /root/.cache/huggingface/stored_tokens
Your token

In [2]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import json
import os
from google.colab import files
from IPython.display import display, FileLink
from datasets import load_dataset
import gc

# loading the model

In [ ]:
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
gc.collect()

NameError: name 'torch' is not defined

In [3]:
model_path = "inceptionai/jais-family-6p7b-chat"

tokenizer = AutoTokenizer.from_pretrained(model_path, use_auth_token=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",  # Automatically maps the model to the GPU
    torch_dtype=torch.float16,  # Use half-precision for faster computation and reduced memory
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

...

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/8.82G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
def get_response(text, do_sample, top_p, num_return_sequences):
    encoding = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=100)
    input_ids = encoding.input_ids.to(model.device)  # Ensure input is sent to the model's device
    attention_mask = encoding.attention_mask.to(model.device)

    generate_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=50,
        num_beams=10,
        top_p=top_p,
        temperature=0.5,
        repetition_penalty=1.2,
        do_sample=do_sample,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decode all generated sequences
    return [tokenizer.decode(ids, skip_special_tokens=True) for ids in generate_ids]


# helper functions

In [5]:
def generate_answer(question, num_samples, model, tokenizer):
    # Create the prompt with the question
    prompt = f" أجب على السؤال التالي بجملة واحدة فقط موجزة ولكن كاملة باللغة العربية\nQuestion: {question}\nAnswer:"

    # Tokenize the prompt without adding special tokens
    prompt_tokens = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").to(model.device)
    prompt_length = prompt_tokens.input_ids.shape[1]  # Get actual prompt length

    # Generate responses using the model
    outputs = model.generate(
        **prompt_tokens,
        max_length=150,
        do_sample=True,
        temperature=0.5,
        top_p=0.95,
        num_return_sequences=num_samples,
        return_dict_in_generate=True,
        output_scores=True,
    )

    sequences = outputs.sequences
    results = []

    for seq in sequences:
        # Decode and clean the generated text
        decoded_text = tokenizer.decode(seq, skip_special_tokens=True)

        # Extract the answer after the "Answer:" marker
        answer = decoded_text.split("Answer:")[-1].strip()

        #clean the output like what they did
        answer = decoded_text.split("Answer:")[-1].strip()
        strings_to_filter_on = ['.', '\n', 'Q:', 'A:', 'question:', 'answer:', 'Question:', 'Answer:', 'Questions:', 'questions:', 'QUESTION:', 'ANSWER:']
        for string in strings_to_filter_on:
            if string in answer:
                answer = answer.split(string)[0].strip()

        # Calculate negative log-likelihood
        with torch.no_grad():
            # Create target labels with prompt masked
            target_ids = seq.clone()
            target_ids[:prompt_length] = -100  # Ignore prompt tokens in loss

            # Forward pass to compute loss
            output = model(seq.unsqueeze(0), labels=target_ids.unsqueeze(0))

            # Calculate total NLL for generated tokens
            avg_nll_per_token = output.loss
            num_generated_tokens = len(seq) - prompt_length
            total_nll = avg_nll_per_token * num_generated_tokens

        results.append({
            'text': answer,
            'neg_log_likelihood': total_nll.item(),
        })

    return results

In [6]:
def load_qa_dataset(dataset_name, file_path=None):
    data = []

    try:
        if dataset_name == 'arabicaqa' and file_path:
            if os.path.exists(file_path):  # Use the loaded file
                print("Using ArabicaQA")
                with open(file_path, 'r', encoding='utf-8') as f:
                    custom_data = json.load(f)
                    for idx, item in enumerate(custom_data):
                        data.append({
                            "question_id": idx,
                            "Question": item["question"],
                            "Answer": item["answer"]
                        })
            else:
                raise FileNotFoundError(f"ArabicaQA file not found at {file_path}")

        elif dataset_name == 'xor_tydiqa' and file_path:
            if os.path.exists(file_path):  # Use the loaded file
                print("Using XOR-TyDiQA")
                print("Filtering Arabic QA pairs from XOR-TyDi...")

                # Load the jsonl dataset
                with open(file_path, 'r', encoding='utf-8') as f:
                    custom_data = [json.loads(line) for line in f]  # Handling jsonl

                # Filter Arabic samples ("lang" = "ar")
                arabic_data = []
                for idx, item in enumerate(custom_data):
                    if item["lang"] == "ar":  # Arabic language code
                        arabic_data.append({
                            "question_id": item["id"],
                            "Question": item["question"],
                            "Answer": item["answers"][0]  # First answer in list
                        })

                data.extend(arabic_data)
            else:
                raise FileNotFoundError(f"XOR-TyDiQA file not found at {file_path}")

        elif dataset_name == 'xquad':
            print("Using XQuAD")
            dataset = load_dataset("xquad", "xquad.ar")
            for idx, item in enumerate(dataset["validation"]):  # Only available set
                data.append({
                    "question_id": f"xquad_{idx}",
                    "Question": item["question"],
                    "Answer": item["answers"]["text"][0]
                })

        elif dataset_name == 'arcd':
            print("Using ARCD")
            dataset = load_dataset("arcd")
            for split in ["train", "validation"]:
                for idx, item in enumerate(dataset[split]):
                    data.append({
                        "question_id": f"arcd_{idx}",
                        "Question": item["question"],
                        "Answer": item["answers"]
                    })

        elif dataset_name == 'arabic_squad':
            print("Using Arabic SQuAD")
            dataset = load_dataset("Mostafa3zazi/Arabic_SQuAD")
            for split in ["train"]:
                for idx, item in enumerate(dataset[split]):
                    data.append({
                        "question_id": f"squad_{idx}",
                        "Question": item["question"],
                        "Answer": item["text"]
                    })

        elif dataset_name == 'mintaka':
            print(f"Using Mintaka in arabic")
            dataset = load_dataset("AmazonScience/mintaka", "ar")
            if "train" in dataset:
                dataset = dataset["train"]  # Select training split
            for idx, item in enumerate(dataset):
                    data.append({
                        "question_id": item['id'],
                        "Question": item['question'],
                        "Answer": item['answerText']
                    })

        elif dataset_name == 'aqad' and file_path:
            if os.path.exists(file_path):
                print("Using AQAD")
                # Load the dataset
                with open(file_path, 'r', encoding='utf-8') as f:
                    dataset = json.load(f)  # Assuming the dataset is in JSON format

                # Loop through the data structure
                for article in dataset.get('data', []):  # Ensure to check 'data'
                    title = article.get('title', 'Unknown Title')  # Handle missing title
                    for paragraph in article.get('paragraphs', []):  # Ensure to check 'paragraphs'
                        for qas in paragraph.get('qas', []):  # Ensure to check 'qas'
                            question = qas.get('question', '').strip()  # Handle missing question
                            answers = qas.get('answers', [])
                            if answers:
                                answer_text = answers[0].get('text', '').strip()  # Handle missing answer
                                if question and answer_text:  # Ensure both question and answer are present
                                    data.append({
                                        "question_id": f"aqad_{qas.get('id', 'unknown')}",  # Handle missing ID
                                        "Question": question,
                                        "Answer": answer_text
                                    })
                            else:
                                print(f"Skipping entry with missing answer for question: {question}")

        else:
            raise ValueError(f"Unsupported dataset: {dataset_name}")

    except Exception as e:
        print(f"Error loading {dataset_name}: {str(e)}")
        return []

    return data

# main function to to load questions and gerate answers and save them to a file

In [7]:
 # Main function
def main():
    # Step 1: SELECT AND LOAD DATA
    DATASET_CHOICE = 'arabicaqa'  # Choose from above

    FILE_PATH = '/content/test-open.json'  # Only needed for ArabicaQA And xor_tydiqa and AAQAD

    data = load_qa_dataset(DATASET_CHOICE, FILE_PATH if DATASET_CHOICE == 'arabicaqa' or 'xor_tydiqa' else None)

    if not data:
        print("No data loaded. Exiting...")
        return

    # Convert to DataFrame
    df = pd.DataFrame(data)[["question_id", "Question", "Answer"]]
    print(f"Loaded {len(df)} questions\n")

  # Step 2: GENERATE ANSWERS
  #empty list to store the results
    results = []

    for index, row in df.iterrows():
        print(f"Processing question {index+1}/{len(df)}")
        question = row['Question']
        original_answer = row.get('Answer',"")  # Use empty string if no original answer provided
        question_id = row.get('question_id')

        # Generate 10 answers with log probabilities for each question
        generated_results = generate_answer(
            question=question,
            num_samples=10,
            model=model,
            tokenizer=tokenizer
        )
        # Extract text answers and log probabilities
        # Append the results

        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": row.get('Answer', ""),
            "generated_answers": [
                {"text": r['text'], "neg_log_likelihood": r['neg_log_likelihood']}
                for r in generated_results
            ]
        })

   # Step 3: SAVE RESULTS
    results_df = pd.DataFrame(results)
    print(results_df)

    filename = f"generated_answers_jais-family-6p7b-chat{DATASET_CHOICE}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    print(f"\nSaved results to {filename}")
    files.download(filename)

# Run the main function
if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
Processing question 7635/12592
Processing question 7636/12592
Processing question 7637/12592
Processing question 7638/12592
Processing question 7639/12592
Processing question 7640/12592
Processing question 7641/12592
Processing question 7642/12592
Processing question 7643/12592
Processing question 7644/12592
Processing question 7645/12592
Processing question 7646/12592
Processing question 7647/12592
Processing question 7648/12592
Processing question 7649/12592
Processing question 7650/12592
Processing question 7651/12592
Processing question 7652/12592
Processing question 7653/12592
Processing question 7654/12592
Processing question 7655/12592
Processing question 7656/12592
Processing question 7657/12592
Processing question 7658/12592
Processing question 7659/12592
Processing question 7660/12592
Processing question 7661/12592
Processing question 7662/12592
Processing question 7663/12592
Processing question 7664/12592
Processing question

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>